In [1]:
from src.data.tools.run_model import segment_images, vis_predictions

In [5]:
%%time
rgbn_2_preds = segment_images(model_name='rgbn', roi=2,
               img_dir='/home/jovyan/lustre_scratch/data_ne/ai_data/val_images',
               mask_dir='/home/jovyan/lustre_scratch/data_ne/ai_data/val_masks')

ndvi_2_preds = segment_images(model_name='ndvi', roi=2,
               img_dir='/home/jovyan/lustre_scratch/data_ne/ai_data/val_images',
               mask_dir='/home/jovyan/lustre_scratch/data_ne/ai_data/val_masks')

fusion_2_preds = segment_images(model_name='fusion', roi=2,
               img_dir='/home/jovyan/lustre_scratch/data_ne/ai_data/val_images',
               mask_dir='/home/jovyan/lustre_scratch/data_ne/ai_data/val_masks')

CPU times: user 22.4 s, sys: 6.78 s, total: 29.1 s
Wall time: 32.5 s


In [8]:
%%time
vis_predictions(preds_fusion=fusion_2_preds, preds_ndvi=ndvi_2_preds,
                preds_rgbn=rgbn_2_preds, roi=2,
                img_dir='/home/jovyan/lustre_scratch/data_ne/ai_data/val_images',
                mask_dir='/home/jovyan/lustre_scratch/data_ne/ai_data/val_masks')

CPU times: user 3min 36s, sys: 13.7 s, total: 3min 50s
Wall time: 3min 43s


In [10]:
%%time
rgbn_1_preds = segment_images(model_name='rgbn', roi=1)

ndvi_1_preds = segment_images(model_name='ndvi', roi=1)

fusion_1_preds = segment_images(model_name='fusion', roi=1)

CPU times: user 1min 19s, sys: 20.9 s, total: 1min 40s
Wall time: 2min 26s


In [11]:
%%time
vis_predictions(preds_fusion=fusion_1_preds, preds_ndvi=ndvi_1_preds,
                preds_rgbn=rgbn_1_preds, roi=1)

CPU times: user 13min 9s, sys: 42.3 s, total: 13min 51s
Wall time: 13min 27s


In [12]:
import zipfile
import os
import glob

def zip_directory(directory_path, zip_path):
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        files = glob.glob(directory_path + '/*.png')
        for file in files:
            file_path = os.path.join(directory_path, file)
            zipf.write(file_path, arcname=os.path.relpath(file_path, directory_path))

# Provide the directory path and the desired zip file path
directory_path = '/home/jovyan/lustre_scratch/reforestation-CNN/data/predictions'
zip_path = '/home/jovyan/lustre_scratch/reforestation-CNN/data/predictions/images_roi_1.zip'

# Call the function to zip the directory
zip_directory(directory_path, zip_path)